<a href="https://colab.research.google.com/github/blaw5/dlwpt-code/blob/master/CNN_TPU_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl tensorboard-plugin-profile

     |████████████████████████████████| 152.9 MB 35 kB/s 
     |████████████████████████████████| 750.6 MB 10 kB/s 
     |████████████████████████████████| 1.1 MB 66.4 MB/s 
     |████████████████████████████████| 57 kB 5.6 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchaud

In [ ]:
from torchvision import datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torch.nn.functional as F
import datetime
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
data_path = "../content/drive/MyDrive/ColabNotebooks/data/hw3/"
cifar10 = datasets.CIFAR10(data_path, train = True, download = True, transform=transforms.ToTensor())
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True, transform=transforms.ToTensor())

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../content/drive/MyDrive/ColabNotebooks/data/hw3/cifar-10-python.tar.gz to ../content/drive/MyDrive/ColabNotebooks/data/hw3/
Files already downloaded and verified


In [ ]:
imgs = torch.stack([img_t for img_t, _ in cifar10], dim=3)
print(imgs.view(3, -1).mean(dim=1))
print(imgs.view(3, -1).std(dim=1))
imgs_v = torch.stack([img_t for img_t, _ in cifar10_val], dim=3)
print(imgs_v.view(3, -1).mean(dim=1))
print(imgs_v.view(3, -1).std(dim=1))

tensor([0.4914, 0.4822, 0.4465])
tensor([0.2470, 0.2435, 0.2616])
tensor([0.4942, 0.4851, 0.4504])
tensor([0.2467, 0.2429, 0.2616])


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8) #need to mod outputs for 10
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = Net()

In [ ]:
def _run(model):
 
  def training_loop(n_epoch, train_loader,optimizer, loss_fn, device):
            model.train()
            running_loss = 0.0
            total = 0.0
            for imgs, labels in train_loader:
                  imgs = imgs.to(device=device)
                  labels = labels.to(device=device)
                  outputs = model(imgs)

                  loss = loss_fn(outputs, labels)
                  optimizer.zero_grad()
                  
                  loss.backward()
                  xm.optimizer_step(optimizer)

                  running_loss += loss.item()
            if n_epoch == 1 or n_epoch % 10 == 0:
                  xm.master_print('EPOCH: {:}'.format(n_epoch))
                  xm.master_print('training Loss: {:.4f}'.format(
                    running_loss/len(train_loader)))

  def validate(epoch, val_loader, loss_fn, device):
    correct = 0
    total = 0

    model.eval()
    for imgs,labels in val_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        outputs = model(imgs)
        loss = loss_fn(outputs, labels)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    if epoch == 1 or epoch % 10 == 0:
      xm.master_print('Accuracy {:.2f}'.format(correct / total))
  EPOCHS = 300
  BATCH_SIZE = 64

  transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465),
                            (0.2470, 0.2435, 0.2616))
  ]))
  transformed_cifar10_val = datasets.CIFAR10(
  data_path, train=False, download=False,
  transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4942, 0.4851, 0.4504),
                          (0.2467, 0.2429, 0.2616))
  ]))
  train_sampler = torch.utils.data.distributed.DistributedSampler(
          transformed_cifar10,
          num_replicas=xm.xrt_world_size(),
          rank=xm.get_ordinal(),
          shuffle=True) 
  valid_sampler = torch.utils.data.distributed.DistributedSampler(
            transformed_cifar10_val,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=False)
  training_dataloader = torch.utils.data.DataLoader(
                        transformed_cifar10,
                        num_workers=4,
                        batch_size=BATCH_SIZE,
                        sampler=train_sampler,
                        drop_last=True
                       )
  val_dataloader = torch.utils.data.DataLoader(
                          transformed_cifar10_val,
                          num_workers=4,
                          batch_size=64,
                          sampler=valid_sampler,
                          drop_last=False
                        )
  device = xm.xla_device()
  model = model.to(device)
  lr = lr = 1e-2
  loss_fn = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr)

  for epoch in range(EPOCHS):

    para_loader = pl.ParallelLoader(training_dataloader, [device])

    training_loop(n_epoch = epoch+1,
                  train_loader=para_loader.per_device_loader(device), 
                  optimizer=optimizer, 
                  loss_fn=loss_fn,
                  device=device,)
    with torch.no_grad():
      
      para_loader = pl.ParallelLoader(val_dataloader, [device])
      
      validate(epoch = epoch+1, 
              val_loader=para_loader.per_device_loader(device), 
              loss_fn=loss_fn, 
              device=device)
      xm.save(
                model.state_dict(),
                f'efficientnet-b0-bs-8.pt'
              )

In [ ]:
def _mp_fn(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    a = _run(model)

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

EPOCH: 1
training Loss: 2.2754
Accuracy 0.18
EPOCH: 10
training Loss: 1.8217
Accuracy 0.39
EPOCH: 20
training Loss: 1.6015
Accuracy 0.47
EPOCH: 30
training Loss: 1.4874
Accuracy 0.51
EPOCH: 40
training Loss: 1.3882
Accuracy 0.55
EPOCH: 50
training Loss: 1.3099
Accuracy 0.58
EPOCH: 60
training Loss: 1.2498
Accuracy 0.59
EPOCH: 70
training Loss: 1.2020
Accuracy 0.60
EPOCH: 80
training Loss: 1.1615
Accuracy 0.61
EPOCH: 90
training Loss: 1.1265
Accuracy 0.62
EPOCH: 100
training Loss: 1.0968
Accuracy 0.63
EPOCH: 110
training Loss: 1.0716
Accuracy 0.62
EPOCH: 120
training Loss: 1.0498
Accuracy 0.63
EPOCH: 130
training Loss: 1.0306
Accuracy 0.64
EPOCH: 140
training Loss: 1.0126
Accuracy 0.64
EPOCH: 150
training Loss: 0.9960
Accuracy 0.64
EPOCH: 160
training Loss: 0.9808
Accuracy 0.64
EPOCH: 170
training Loss: 0.9655
Accuracy 0.64
EPOCH: 180
training Loss: 0.9511
Accuracy 0.65
EPOCH: 190
training Loss: 0.9375
Accuracy 0.66
EPOCH: 200
training Loss: 0.9243
Accuracy 0.66
EPOCH: 210
training Loss